# Imports

In [2]:
import streamlit
# import plotly.express as px

import sqlite3 as db
import pandas as pd
import os
import json
import random
import re

import numpy as np
import matplotlib.pyplot as plt

# from IPython.display import display 

import warnings
warnings.filterwarnings("ignore")

# Data

In [3]:
# Hacer una query SQL
def sql_query(q):
    conn = db.connect('../db/sqlite/eicu_v2_0_1_clean.sqlite3')
    df = pd.read_sql_query(q, conn)
    conn.close()
    
    return df

# Leer todos los CSV que hemos descomprimido
def read_csvs():
   

    datasets = [ 'admissiondrug', 'admissionDx', 'allergy', 'apacheApsVar', 'apachePatientResult', 'apachePredVar', 'carePlanCareProvider', 'carePlanEOL', 'carePlanGeneral',
                 'carePlanGoal', 'carePlanInfectiousDisease', 'customLab', 'diagnosis', 'hospital', 'infusiondrug', 'intakeOutput', 'lab', 'medication', 'microLab', 'note',
                 'nurseAssessment', 'nurseCare', 'nurseCharting', 'pastHistory', 'patient', 'physicalExam', 'respiratoryCare', 'respiratoryCharting', 'treatment', 'vitalAperiodic',
                 'vitalPeriodic']

    dfs = {}

    for ds_name in datasets:
        dfs[ds_name] = pd.read_csv('../db/csv_clean/' + ds_name + '.csv')
    
    return dfs

dfs = read_csvs()

In [4]:
dfs.keys()

dict_keys(['admissiondrug', 'admissionDx', 'allergy', 'apacheApsVar', 'apachePatientResult', 'apachePredVar', 'carePlanCareProvider', 'carePlanEOL', 'carePlanGeneral', 'carePlanGoal', 'carePlanInfectiousDisease', 'customLab', 'diagnosis', 'hospital', 'infusiondrug', 'intakeOutput', 'lab', 'medication', 'microLab', 'note', 'nurseAssessment', 'nurseCare', 'nurseCharting', 'pastHistory', 'patient', 'physicalExam', 'respiratoryCare', 'respiratoryCharting', 'treatment', 'vitalAperiodic', 'vitalPeriodic'])

In [5]:
for keys in dfs.keys():
    print(f"###### Processing {keys}")
    print(dfs[keys].info())

###### Processing admissiondrug
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551 entries, 0 to 550
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   patientunitstayid  551 non-null    int64
 1   aspirin            551 non-null    bool 
 2   lisinopril         551 non-null    bool 
 3   lasix              551 non-null    bool 
dtypes: bool(3), int64(1)
memory usage: 6.0 KB
None
###### Processing admissionDx
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2221 entries, 0 to 2220
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   patientunitstayid      2221 non-null   int64
 1   OffSet_Cardiovascular  2221 non-null   int64
 2   OffSet_Respiratory     2221 non-null   int64
 3   OffSet_Neurologic      2221 non-null   int64
dtypes: int64(4)
memory usage: 69.5 KB
None
###### Processing allergy
<class 'pandas.core.frame.Data

Aunque el dataset original fue limpiado a conciencia, todavía se pueden apreciar muchas tablas interesantes para sacar información. Son tantas <br>
las posibilidades de plots que parecen ilimitadas, así que de manera intuitiva dado el caso de uso intentaré limitar el alcance de la visualización al <br> máximo intentando centrarme en lo más importante. Es posible que durante el proceso me deje algún plot interesante aunque intentaré evitarlo en la medida de lo posible.

## Patient

Como hemos visto durante el proceso de limpieza, `patient` es una de las tablas más interesantes.

In [6]:
dfs['patient'].head()

,patientunitstayid,gender,age,ethnicity,hospitalid,apacheadmissiondx,admissionheight,hospitaladmitoffset,hospitaladmitsource,hospitaldischargeoffset,unitvisitnumber,admissionweight,unitdischargeoffset,avg_unit_stay,avg_hospital_stay,admission_bmi
0,141764,Female,87,Caucasian,59,NaN,157.5,-2258,Emergency Department,366,2,67.6,344,0,0,-1.000000
1,141765,Female,87,Caucasian,59,"rhythm disturbance (atrial, supraventricular)",157.5,-8,Emergency Department,2616,1,46.5,2250,344,366,3.387097
2,143870,Male,76,Caucasian,68,"endarterectomy, carotid",167.0,-1,Operating Room,1218,1,77.5,793,0,0,2.154839
3,144815,Female,34,Caucasian,56,"overdose, other toxin, poison or drug",172.7,-23,Emergency Department,1138,1,60.3,1121,0,0,2.864013
4,145427,Male,61,Caucasian,68,"gi perforation/rupture, surgery for",177.8,-10,Emergency Department,5263,1,91.7,1369,0,0,1.938931


In [7]:
dfs['patient'].columns

Index(['patientunitstayid', 'gender', 'age', 'ethnicity', 'hospitalid',
       'apacheadmissiondx', 'admissionheight', 'hospitaladmitoffset',
       'hospitaladmitsource', 'hospitaldischargeoffset', 'unitvisitnumber',
       'admissionweight', 'unitdischargeoffset', 'avg_unit_stay',
       'avg_hospital_stay', 'admission_bmi'],
      dtype='object')

### Plots

In [8]:
# ---------- Preprocesado ----------
def numeric_age(s: pd.Series) -> pd.Series:
    """
    Convierte 'age' a años numéricos.
    Maneja casos como '> 89' -> 90.
    """
    s = s.astype(str)
    older_mask = s.str.contains(">")
    out = pd.to_numeric(s.str.extract(r"(\d+)")[0], errors="coerce")
    # Si venía como '> 89' y no se pudo extraer un número, asumimos 90.
    out[older_mask] = np.where(out[older_mask].notna(), out[older_mask], 90)
    return out

def preprocess_patient(df: pd.DataFrame) -> pd.DataFrame:
    """
    Crea columnas útiles:
      - age_years
      - icu_los_days = unitdischargeoffset / (60*24)
    Limpia ligeramente categorías.
    """
    df = df.copy()

    # Edad
    if "age" in df.columns:
        df["age_years"] = numeric_age(df["age"])
    else:
        df["age_years"] = np.nan

    # Estancia UCI en días (unitdischargeoffset está en minutos)
    if "unitdischargeoffset" in df.columns:
        df["icu_los_days"] = pd.to_numeric(df["unitdischargeoffset"], errors="coerce") / (60*24)
    else:
        df["icu_los_days"] = np.nan

    # Rango razonable para gráficos (evita outliers extremos que rompen ejes)
    df = df[(df["icu_los_days"].between(0, 120)) | df["icu_los_days"].isna()]
    df = df[(df["age_years"].between(0, 110)) | df["age_years"].isna()]

    # Limpieza estética de texto
    for c in ["gender", "ethnicity", "hospitaladmitsource", "apacheadmissiondx"]:
        if c in df.columns:
            df[c] = df[c].astype("string").str.strip().str.title()

    # IDs como string (por si hay NaN/Int64)
    for c in ["patientunitstayid", "hospitalid"]:
        if c in df.columns:
            df[c] = df[c].astype("Int64").astype("string")

    return df

# ---------- Helpers ----------
def top_n_index(s: pd.Series, n: int) -> pd.Index:
    """Devuelve las etiquetas top-n por conteo."""
    return s.value_counts(dropna=False).head(n).index

# ---------- PLOTS ----------
def plot_hist_los(df: pd.DataFrame, bins: int = 50, logy: bool = False):
    """1) Histograma de estancia UCI (días)."""
    
    x = df["icu_los_days"].dropna()
    
    fig, ax = plt.subplots(figsize=(7,4.5))
    
    ax.hist(x, bins=50, edgecolor="black")
    ax.set_title("Histograma de estancia en UCI (días)")
    ax.set_xlabel("Estancia UCI (días)")
    ax.set_ylabel("Frecuencia")
    ax.grid(True, linestyle="--", alpha=0.4)
    
    plt.tight_layout()
    return fig, ax

def plot_box_by_gender(df: pd.DataFrame):
    """2) Boxplot de estancia por género."""
    if "gender" not in df.columns:
        raise ValueError("Falta la columna 'gender'.")
    data = [df.loc[df["gender"]==g, "icu_los_days"].dropna() 
            for g in df["gender"].dropna().unique()]
    labels = list(df["gender"].dropna().unique())

    fig, ax = plt.subplots(figsize=(7,4.5))
    ax.boxplot(data, labels=labels, showfliers=True)
    ax.set_title("Estancia en UCI por género")
    ax.set_xlabel("Género")
    ax.set_ylabel("Estancia UCI (días)")
    ax.grid(True, linestyle="--", alpha=0.4)
    plt.tight_layout()
    return fig, ax

def plot_bar_admit_source(df: pd.DataFrame, top_n: int = 12):
    """3) Barras: fuentes de admisión hospitalaria (top N por conteo)."""
    if "hospitaladmitsource" not in df.columns:
        raise ValueError("Falta la columna 'hospitaladmitsource'.")
    counts = df["hospitaladmitsource"].fillna("Desconocido").value_counts().head(top_n)

    fig, ax = plt.subplots(figsize=(8,5))
    ax.bar(counts.index, counts.values, edgecolor="black")
    ax.set_title(f"Fuente de admisión hospitalaria (top {top_n})")
    ax.set_xlabel("Fuente de admisión")
    ax.set_ylabel("Pacientes")
    ax.grid(True, axis="y", linestyle="--", alpha=0.4)
    plt.setp(ax.get_xticklabels(), rotation=25, ha="right")
    plt.tight_layout()
    return fig, ax

def plot_scatter_age_los(df: pd.DataFrame, max_points: int = 50000):
    """4) Dispersión edad vs estancia (downsample si hay demasiados puntos)."""
    tmp = df[["age_years", "icu_los_days"]].dropna()
    if len(tmp) > max_points:
        tmp = tmp.sample(max_points, random_state=42)

    fig, ax = plt.subplots(figsize=(7,4.5))
    ax.scatter(tmp["age_years"], tmp["icu_los_days"], alpha=0.5, marker="x")
    ax.set_title("Edad vs estancia en UCI")
    ax.set_xlabel("Edad (años)")
    ax.set_ylabel("Estancia UCI (días)")
    ax.grid(True, linestyle="--", alpha=0.4)
    plt.tight_layout()
    return fig, ax

def plot_bar_median_by_dx(df: pd.DataFrame, top_n: int = 10):
    """5) Mediana de estancia por diagnóstico APACHE (top N por volumen)."""
    if "apacheadmissiondx" not in df.columns:
        raise ValueError("Falta la columna 'apacheadmissiondx'.")
    top = top_n_index(df["apacheadmissiondx"], n=top_n)
    tmp = df[df["apacheadmissiondx"].isin(top)].copy()
    med = (tmp.groupby("apacheadmissiondx")["icu_los_days"]
              .median().sort_values(ascending=True))

    fig, ax = plt.subplots(figsize=(9,5))
    ax.barh(med.index, med.values, edgecolor="black")
    ax.set_title(f"Mediana de estancia por diagnóstico (top {top_n})")
    ax.set_xlabel("Mediana estancia (días)")
    ax.grid(True, axis="x", linestyle="--", alpha=0.4)
    plt.tight_layout()
    return fig, ax

def plot_bar_median_by_hospital(df: pd.DataFrame, top_n: int = 10):
    """6) Mediana de estancia por hospital (top N por volumen)."""
    if "hospitalid" not in df.columns:
        raise ValueError("Falta la columna 'hospitalid'.")
    top = top_n_index(df["hospitalid"], n=top_n)
    tmp = df[df["hospitalid"].isin(top)].copy()
    med = (tmp.groupby("hospitalid")["icu_los_days"]
              .median().sort_values(ascending=False))

    fig, ax = plt.subplots(figsize=(8,5))
    ax.bar(med.index.astype(str), med.values, edgecolor="black")
    ax.set_title(f"Mediana de estancia por hospital (top {top_n})")
    ax.set_xlabel("Hospital ID")
    ax.set_ylabel("Mediana estancia (días)")
    ax.grid(True, axis="y", linestyle="--", alpha=0.4)
    plt.tight_layout()
    return fig, ax


In [9]:
df_patient = preprocess_patient(dfs['patient'])
df_patient.head()

,patientunitstayid,gender,age,ethnicity,hospitalid,apacheadmissiondx,admissionheight,hospitaladmitoffset,hospitaladmitsource,hospitaldischargeoffset,unitvisitnumber,admissionweight,unitdischargeoffset,avg_unit_stay,avg_hospital_stay,admission_bmi,age_years,icu_los_days
0,141764,Female,87,Caucasian,59,<NA>,157.5,-2258,Emergency Department,366,2,67.6,344,0,0,-1.000000,87,0.238889
1,141765,Female,87,Caucasian,59,"Rhythm Disturbance (Atrial, Supraventricular)",157.5,-8,Emergency Department,2616,1,46.5,2250,344,366,3.387097,87,1.562500
2,143870,Male,76,Caucasian,68,"Endarterectomy, Carotid",167.0,-1,Operating Room,1218,1,77.5,793,0,0,2.154839,76,0.550694
3,144815,Female,34,Caucasian,56,"Overdose, Other Toxin, Poison Or Drug",172.7,-23,Emergency Department,1138,1,60.3,1121,0,0,2.864013,34,0.778472
4,145427,Male,61,Caucasian,68,"Gi Perforation/Rupture, Surgery For",177.8,-10,Emergency Department,5263,1,91.7,1369,0,0,1.938931,61,0.950694


In [10]:
x = pd.to_numeric(df_patient["icu_los_days"], errors="coerce").dropna()

fig, ax = plt.subplots(figsize=(7, 4.5))
ax.hist(x, bins=50, edgecolor="black")
ax.set_title("Histograma de estancia en UCI (días)")
ax.set_xlabel("Estancia UCI (días)")
ax.set_ylabel("Frecuencia")
ax.grid(True, linestyle="--", alpha=0.4)
plt.tight_layout()
plt.show()

In [11]:
"""1) Histograma de estancia UCI (días)."""

x = df_patient["icu_los_days"].dropna()
x.plot

In [25]:
"""1) Histograma de estancia UCI (días)."""

x = df_patient["icu_los_days"].dropna()

fig, ax = plt.subplots(figsize=(7,4.5))

ax.hist(x, bins=50, edgecolor="black")
ax.set_title("Histograma de estancia en UCI (días)")
ax.set_xlabel("Estancia UCI (días)")
ax.set_ylabel("Frecuencia")
ax.grid(True, linestyle="--", alpha=0.4)

plt.tight_layout()

In [9]:
plot_hist_los(df_patient)

(<Figure size 700x450 with 1 Axes>,
 <Axes: title={'center': 'Histograma de estancia en UCI (días)'}, xlabel='Estancia UCI (días)', ylabel='Frecuencia'>)

In [10]:
plot_box_by_gender(df_patient)

(<Figure size 700x450 with 1 Axes>,
 <Axes: title={'center': 'Estancia en UCI por género'}, xlabel='Género', ylabel='Estancia UCI (días)'>)

In [11]:
plot_bar_admit_source(df_patient)

(<Figure size 800x500 with 1 Axes>,
 <Axes: title={'center': 'Fuente de admisión hospitalaria (top 12)'}, xlabel='Fuente de admisión', ylabel='Pacientes'>)

In [12]:
plot_scatter_age_los(df_patient)

(<Figure size 700x450 with 1 Axes>,
 <Axes: title={'center': 'Edad vs estancia en UCI'}, xlabel='Edad (años)', ylabel='Estancia UCI (días)'>)

In [13]:
plot_bar_median_by_dx(df_patient)

(<Figure size 900x500 with 1 Axes>,
 <Axes: title={'center': 'Mediana de estancia por diagnóstico (top 10)'}, xlabel='Mediana estancia (días)'>)

In [14]:
plot_bar_median_by_hospital(df_patient)

(<Figure size 800x500 with 1 Axes>,
 <Axes: title={'center': 'Mediana de estancia por hospital (top 10)'}, xlabel='Hospital ID', ylabel='Mediana estancia (días)'>)